In [12]:
import sys
import os
from dotenv import load_dotenv

import numpy as np
import pandas as pd
import sqlalchemy as sq

sys.path.append("../Shared/")
from DataService import DataService

# fetch data from db

In [5]:
# function to update logs
def updateLog(fileName: str, message: str) -> None:
    try:
        if fileName is not None:
            with open(fileName, "a") as log:
                log.write(message + "\n")
    except Exception as e:
        print(message)

In [10]:
LOG_FILE = "/data/pull_moisture.log"

load_dotenv()
PG_USER = os.getenv("POSTGRES_USER")
PG_PW = os.getenv("POSTGRES_PW")
PG_DB = os.getenv("POSTGRES_DB")
PG_ADDR = os.getenv("POSTGRES_ADDR")
PG_PORT = os.getenv("POSTGRES_PORT")

In [11]:
if (
    PG_DB is None
    or PG_ADDR is None
    or PG_PORT is None
    or PG_USER is None
    or PG_PW is None
):
    updateLog(LOG_FILE, "Missing database credentials")
    raise ValueError("Environment variables are not set")
else:
    # connicting to database
    db = DataService(PG_DB, PG_ADDR, int(PG_PORT), PG_USER, PG_PW)
    conn = db.connect()

In [14]:
# pulling soil moisture data
query = sq.text("select * FROM public.agg_soil_moisture")
sm_df = pd.read_sql(query, conn)

In [16]:
sm_df.head()

,index,year,month,day,cr_num,district,soil_moisture_min,soil_moisture_max,soil_moisture_mean
0,0,1978,11,1,0,4612,0.165900,0.418710,0.237053
1,1,1978,11,1,5,4740,0.127140,0.207248,0.163722
2,2,1978,11,1,5,4741,0.153398,0.215304,0.170989
3,3,1978,11,1,7,4770,0.140772,0.186390,0.152211
4,4,1978,11,1,7,4771,0.116810,0.116810,0.116810


In [15]:
# pull ergot data
query = sq.text("select * FROM public.agg_ergot_sample")
ergot_df = pd.read_sql(query, conn)

In [17]:
ergot_df.head()

,sample_id,year,province,crop_district,incidence,severity,district,percnt_true,has_ergot,sum_severity,present_prev1,present_prev2,present_prev3,present_in_neighbor,severity_prev1,severity_prev2,severity_prev3,severity_in_neighbor
0,1,1995,AB,1,False,0.0,4810,0.0,False,0.0,False,False,False,False,0.0,0.0,0.0,0.0
1,2,1995,AB,1,False,0.0,4810,0.0,False,0.0,False,False,False,False,0.0,0.0,0.0,0.0
2,3,1995,AB,1,False,0.0,4810,0.0,False,0.0,False,False,False,False,0.0,0.0,0.0,0.0
3,4,1995,AB,1,False,0.0,4810,0.0,False,0.0,False,False,False,False,0.0,0.0,0.0,0.0
4,5,1995,AB,1,False,0.0,4810,0.0,False,0.0,False,False,False,False,0.0,0.0,0.0,0.0


In [ ]:
# pull weather data